# API Distribution

In [31]:
import json
import re
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random

In [11]:
with open('../train.json', 'r') as file:
    data = json.load(file)

In [12]:
conversations = []
for indexed_data in data:
    conversations.append([sentence for sentence in indexed_data['conversations']])

In [13]:
ask_response_pairs = []
for conversation_data in conversations:
    for i in range(len(conversation_data) - 1):
        if conversation_data[i]["from"] == "assistant" and conversation_data[i + 1]["from"] == "function":
            pair = (conversation_data[i]["value"], conversation_data[i + 1]["value"])
            ask_response_pairs.append(pair)

In [14]:
len(ask_response_pairs)

53211

In [15]:
api_responses = []
for ask, respond in ask_response_pairs:
    pattern = r'\nAction: ([^\n]+)'
    api = re.search(pattern, ask).group(1)
#     filtered_apis = [string for string in full_apis if string != 'Finish']
    api = api.split('_for_', 1)[1] if '_for_' in api else api
    api = api.replace("_", "-")
#     try:
#         parsed_json = json.loads(respond)
#         api_response.append((api, parsed_json))
#         print("JSON parsed successfully:", parsed_json)
#     except json.JSONDecodeError as e:
#         print("JSON decoding failed:", e)
    if respond.startswith('{"error": ""'):
        response_index = respond.find("response")
        response_string = respond[response_index + 10:]
        api_responses.append((api, response_string))

In [16]:
api_responses

[('memeados',
  ' "\\ufffdPNG\\r\\n\\u001a\\n\\u0000\\u0000\\u0000\\rIHDR\\u0000\\u0000\\u0002z\\u0000\\u0000\\u0002\\ufffd\\b\\u0006\\u0000\\u0000\\u0000S\\ufffdK\\ufffd\\u0000\\u0000\\u0000\\u0006bKGD\\u0000\\ufffd\\u0000\\ufffd\\u0000\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\u0000\\u0000 \\u0000IDATx\\ufffd\\ufffdM\\ufffd,\\u02d2\\ufffd\\ufffd\\ufffd\\ufffdGd\\ufffd}N\\u001d\\ufffd\\uebaa\\u0016*\\ufffd\\ufffdF\\ufffda\\u02901\\ufffd\\u00131\\ufffd_0c\\ufffd\\u0018 A\\u000f\\u001a\\ufffdF\\u0002\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\u001f\\ufffd\\ufffd2\\ufffd\\u074c\\ufffd\\ufffdGD\\ufffdZk\\ufffd{\\u03ad\\ufffd\\ufffd-\\ufffd\\ufffd\\u0731232\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd>\\\\\\ufffd\\u001e\\ufffd\\u0000\\n!\\ufffd\\u001e\\b\\u001d\\r\\u0005\\u0002\\ufffd\\u0000\\u0000s\\u0010ps\\u0002p\\u0000\\u0014\\ufffd\\ufffd\\ufffdl\\ufffdYD\\ufffd\\ufffd\\ufffd\\ufffd}c~\\ufffd\\ufffd7\\u0011\\u0001RW\\u0010%\\b\\"\\u001c\\ufffd\\ufffdL\\u0015\\ufffd\\u0000\\ufffd\\ufffd\\ufffd\

In [17]:
file_path = 'api_data.csv'
api_data = pd.read_csv(file_path)

In [18]:
popularity = api_data.iloc[:, [-4, -3]]
new_column_labels = ['API', 'Popularity']
popularity.columns = new_column_labels

In [19]:
indices_above_6 = popularity[popularity['Popularity'] > 6].index

In [20]:
api_data = api_data.loc[indices_above_6].iloc[:, [-4, -2]]
new_column_labels = ['API', 'Delay']
api_data.columns = new_column_labels

In [21]:
mapping_dict = api_data.set_index("API")['Delay'].to_dict()

In [22]:
# check proportion

In [23]:
cleaned_delays = [num for num in api_data["Delay"] if num == num]
converted_delays = [int(delay.replace(' ms', '').replace(',', '')) for delay in cleaned_delays]

In [25]:
api_responses

[('memeados',
  ' "\\ufffdPNG\\r\\n\\u001a\\n\\u0000\\u0000\\u0000\\rIHDR\\u0000\\u0000\\u0002z\\u0000\\u0000\\u0002\\ufffd\\b\\u0006\\u0000\\u0000\\u0000S\\ufffdK\\ufffd\\u0000\\u0000\\u0000\\u0006bKGD\\u0000\\ufffd\\u0000\\ufffd\\u0000\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\u0000\\u0000 \\u0000IDATx\\ufffd\\ufffdM\\ufffd,\\u02d2\\ufffd\\ufffd\\ufffd\\ufffdGd\\ufffd}N\\u001d\\ufffd\\uebaa\\u0016*\\ufffd\\ufffdF\\ufffda\\u02901\\ufffd\\u00131\\ufffd_0c\\ufffd\\u0018 A\\u000f\\u001a\\ufffdF\\u0002\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\u001f\\ufffd\\ufffd2\\ufffd\\u074c\\ufffd\\ufffdGD\\ufffdZk\\ufffd{\\u03ad\\ufffd\\ufffd-\\ufffd\\ufffd\\u0731232\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd\\ufffd>\\\\\\ufffd\\u001e\\ufffd\\u0000\\n!\\ufffd\\u001e\\b\\u001d\\r\\u0005\\u0002\\ufffd\\u0000\\u0000s\\u0010ps\\u0002p\\u0000\\u0014\\ufffd\\ufffd\\ufffdl\\ufffdYD\\ufffd\\ufffd\\ufffd\\ufffd}c~\\ufffd\\ufffd7\\u0011\\u0001RW\\u0010%\\b\\"\\u001c\\ufffd\\ufffdL\\u0015\\ufffd\\u0000\\ufffd\\ufffd\\ufffd\

### Mapping & Generating Dist

In [36]:
data = {}

for s, answer in api_responses:
    for key in mapping_dict:
        if set(s).issubset(set(key)):
            delay = mapping_dict[key]
            if s not in data.keys():
                data[s] = [int(delay.replace(' ms', '').replace(',', '')), answer]
            else:
                data[s].append(answer)
#                 print("xxx")
            break

print(data)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [38]:
data_random = {}
for key in data.keys():
    entry = data[key]
    response = random.choice(entry[1:])
    data_random[key] = [entry[0], response]

In [39]:
data_random

{'memeados': [681, ' "Internal Server Error"}'],
 'instagram-api': [134,
  ' "{\'message\': \'This endpoint is disabled for your subscription\'}"}'],
 'the-cocktail-db': [114,
  ' "[{\\"id\\":\\"1\\",\\"title\\":\\"Nutella and cognac coffee cocktail\\",\\"difficulty\\":\\"Easy\\",\\"image\\":\\"https://apipics.s3.amazonaws.com/coctails_api/1.jpg\\"},{\\"id\\":\\"2\\",\\"title\\":\\"Easy rhubarb cordial\\",\\"difficulty\\":\\"Easy\\",\\"image\\":\\"https://apipics.s3.amazonaws.com/coctails_api/2.jpg\\"},{\\"id\\":\\"3\\",\\"title\\":\\"Bottled chocolate orange negroni\\",\\"difficulty\\":\\"Easy\\",\\"image\\":\\"https://apipics.s3.amazonaws.com/coctails_api/3.jpg\\"},{\\"id\\":\\"4\\",\\"title\\":\\"Pickled bramble martini\\",\\"difficulty\\":\\"Easy\\",\\"image\\":\\"https://apipics.s3.amazonaws.com/coctails_api/4.jpg\\"},{\\"id\\":\\"5\\",\\"title\\":\\"Toasted marshmallow hot-buttered rum\\",\\"difficulty\\":\\"Easy\\",\\"image\\":\\"https://apipics.s3.amazonaws.com/coctails_api/5.j

In [40]:
csv_file_path = './api_response_data.csv'
import csv
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['API', 'Delay', 'Responses'])
    for key, (delay, response) in data_random.items():
        writer.writerow([key, delay, response])

csv_file_path

'./api_response_data.csv'